In [1]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP/project

Mounted at /gdrive
/gdrive/My Drive/NLP/project


In [103]:
%pip install transformers datasets rouge_score

In [4]:
!nvidia-smi

Thu Nov 11 22:22:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [37]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn import functional as F

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

from transformers import GPT2LMHeadModel,  GPT2TokenizerFast, GPT2Config, DataCollatorForLanguageModeling, LineByLineTextDataset
from transformers import AdamW, top_k_top_p_filtering
from datasets import load_metric

from tqdm import tqdm
import nltk

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training

## Elon Musk

In [ ]:
!python run_language_modeling.py --output_dir=models/elonmusk --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/elonmusk/train.txt --eval_data_file=data/elonmusk/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

10/26/2021 02:47:04 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:47:04 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Andrej Karpathy

In [ ]:
!python run_language_modeling.py --output_dir=models/karpathy --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/karpathy/train.txt --eval_data_file=data/karpathy/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

10/26/2021 02:16:07 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:16:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Barack Obama

In [ ]:
!python run_language_modeling.py --output_dir=models/BarackObama --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/BarackObama/train.txt --eval_data_file=data/BarackObama/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=3

10/26/2021 02:26:06 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:26:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## WSJ

In [ ]:
!python run_language_modeling.py --output_dir=models/wsj --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/wsj/train.txt --eval_data_file=data/wsj/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=2 --save_strategy=epoch

10/26/2021 03:16:38 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 03:16:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## CNN

In [ ]:
!python run_language_modeling.py --output_dir=models/cnn --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/cnn/train.txt --eval_data_file=data/cnn/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=2 --save_strategy=epoch

10/26/2021 03:40:09 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 03:40:09 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

# Generation and Metrics

In [105]:
def compute_bleu_score(model, dataset):
    bleu = load_metric('bleu')
    with torch.no_grad():
        for i, input in enumerate(test_dataloader):
            input = input.to(device)
            outputs = model(**input)
            last_layer_logits = outputs.logits[:, -1, :]
            top_logits = top_k_top_p_filtering(last_layer_logits, top_k=50, top_p=1.0)
            probabilities = F.softmax(top_logits, dim=-1)
            generated_next_token = torch.multinomial(probabilities, num_samples=1)
            predictions = torch.cat([input.input_ids, generated_next_token], dim=-1).squeeze()
            references = input.input_ids.squeeze()
            bleu.add_batch(predictions=[generated_ids], references=[[input_ids]])
    return bleu.compute()

def compute_rouge_score(model, dataset):
    rouge = load_metric("rouge")
    with torch.no_grad():
        for i, input in enumerate(test_dataloader):
            input = input.to(device)
            outputs = model(**input)
            last_layer_logits = outputs.logits[:, -1, :]
            top_logits = top_k_top_p_filtering(last_layer_logits, top_k=50, top_p=1.0)
            probabilities = F.softmax(top_logits, dim=-1)
            generated_next_token = torch.multinomial(probabilities, num_samples=1)
            predictions = torch.cat([input.input_ids, generated_next_token], dim=-1).squeeze()
            references = input.input_ids.squeeze()
            rouge.add_batch(predictions=[generated_ids], references=[input_ids])
    return rouge.compute()

## Elon Musk

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/elonmusk" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:28 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Am excited about the Model 3 unveil and road trip this weekend. There are some huge changes coming to the design as well, so this one is easy & fast. 
=== GENERATED SEQUENCE 2 ===
To be clear, I am against artificial intelligence. AI should not be in charge. I want it to be. 
=== GENERATED SEQUENCE 3 ===
The last remaining soul mate of the late King Arthur was actually Dragan Motavalli. He was one of the greats. 
=== GENERATED SEQUENCE 4 ===
My kids love  @SpaceX  and  @NASA. 
=== GENERATED SEQUENCE 5 ===
It's not just me: I have several cats. I also own a chicken

In [106]:
model = GPT2LMHeadModel.from_pretrained('models/elonmusk').to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('models/elonmusk')
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
test_dataset = LineByLineTextDataset(tokenizer, 'data/elonmusk/test.txt', tokenizer.model_max_length)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [101]:
bleu_score = compute_bleu_score(model, test_dataloader)
bleu_score['bleu']

0.9621954581957615

In [108]:
rouge_score = compute_rouge_score(model, test_dataloader)
rouge_score['rouge1'].mid

Score(precision=0.9642857142857175, recall=1.0, fmeasure=0.9818181818181791)

## Andrej Karpathy

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/karpathy" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:40 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
"How to Start a Startup" from  @OpenAI  great resource, tips & tricks! 
=== GENERATED SEQUENCE 2 ===
A new Deep Learning tutorial for beginners:  the DeepMind GTC231n video   (w)’s A New Challenge for Computer Visionists 
=== GENERATED SEQUENCE 3 ===
The Deep Web is a Hotbed of Violence and Corruption  soooo excited! 
=== GENERATED SEQUENCE 4 ===
#cvpr2016  a week of hacking on the ConvNetCVPR code. Looks like I'm on a roll with the demos 
=== GENERATED SEQUENCE 5 ===
It's quite amusing to watch what is happening behind closed doors with no public hearings or publ

In [109]:
model = GPT2LMHeadModel.from_pretrained('models/karpathy').to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('models/karpathy')
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
test_dataset = LineByLineTextDataset(tokenizer, 'data/karpathy/test.txt', tokenizer.model_max_length)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [110]:
bleu_score = compute_bleu_score(model, test_dataloader)
bleu_score['bleu']

0.9621954581957615

In [111]:
rouge_score = compute_rouge_score(model, test_dataloader)
rouge_score['rouge1'].mid

Score(precision=0.9642857142857187, recall=1.0, fmeasure=0.9818181818181837)

## Barack Obama

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/BarackObama" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:52 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/BarackObama', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
“They’re going to try to scare us off education entirely, because no one wants to go to college.” 
=== GENERATED SEQUENCE 2 ===
"That said, I know that people can still make their voices heard in Washington. I know it's not easy to do that." —President Obama 
=== GENERATED SEQUENCE 3 ===
The Senate must make a timely vote on an American Jobs Act now. The Jobs Act needs to be included in any deal. 
=== GENERATED SEQUENCE 4 ===
There's still more work to do, but this is progress. 
=== GENERATED SEQUENCE 5 ===
It's up to us to work with leaders in both parties to 

In [112]:
model = GPT2LMHeadModel.from_pretrained('models/BarackObama').to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('models/BarackObama')
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
test_dataset = LineByLineTextDataset(tokenizer, 'data/BarackObama/test.txt', tokenizer.model_max_length)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [113]:
bleu_score = compute_bleu_score(model, test_dataloader)
bleu_score['bleu']

0.9621954581957615

In [114]:
rouge_score = compute_rouge_score(model, test_dataloader)
rouge_score['rouge1'].mid

Score(precision=0.9642857142857107, recall=1.0, fmeasure=0.9818181818181941)

## CNN

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/cnn" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 04:04:59 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/cnn', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Obama: The "threat" to national security from North Korea is not unique, "from my conversations with all my colleagues, from this administration, and from the President himself" 
=== GENERATED SEQUENCE 2 ===
A new study finds that babies born with a rare form of brain cancer can develop better health than their non-nuclear counterparts 
=== GENERATED SEQUENCE 3 ===
The Senate has passed a resolution to censure President Trump for calling former President Joe Biden a 'narcissistic prick' 
=== GENERATED SEQUENCE 4 ===
 @HillaryClinton  and Donald Trump campaign to attend

In [115]:
model = GPT2LMHeadModel.from_pretrained('models/cnn').to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('models/cnn')
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
test_dataset = LineByLineTextDataset(tokenizer, 'data/cnn/test.txt', tokenizer.model_max_length)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [116]:
bleu_score = compute_bleu_score(model, test_dataloader)
bleu_score['bleu']

0.9621954581957615

In [117]:
rouge_score = compute_rouge_score(model, test_dataloader)
rouge_score['rouge1'].mid

Score(precision=0.9642857142857498, recall=1.0, fmeasure=0.9818181818181965)

## WSJ

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/wsj" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 04:05:10 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/wsj', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
‘They’re going to attack again, and it’s going to be a real bloodbath,’ says one Republican in Tennessee. ‘It’s going to be devastating.’ <
=== GENERATED SEQUENCE 2 ===
A new study finds that the number of overweight people is rising.  via  @WSJNY 
=== GENERATED SEQUENCE 3 ===
The Senate tax bill was expected to pass Thursday, but Democrats are delaying key provisions 
=== GENERATED SEQUENCE 4 ===
Opinion: Why the Supreme Court hasn’t heard the case on gay marriage. writes  @wjmcgurn 
=== GENERATED SEQUENCE 5 ===
How to be more ambitious: 


In [118]:
model = GPT2LMHeadModel.from_pretrained('models/wsj').to(device)
tokenizer = GPT2TokenizerFast.from_pretrained('models/wsj')
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
test_dataset = LineByLineTextDataset(tokenizer, 'data/wsj/test.txt', tokenizer.model_max_length)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=data_collator)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [119]:
bleu_score = compute_bleu_score(model, test_dataloader)
bleu_score['bleu']

0.9621954581957615

In [120]:
rouge_score = compute_rouge_score(model, test_dataloader)
rouge_score['rouge1'].mid

Score(precision=0.9642857142857482, recall=1.0, fmeasure=0.9818181818181964)